In [1]:
# Use autoplot 207's code for this app
import sys
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_dbconn, utc
import numpy as np

In [10]:
STORM_NUMBER = 11
TITLE = "14-15 January 2021"
SUBTITLE = "8 AM 16 January 2021"
sts = utc(2021, 1, 15, 10)
ets = utc(2021, 12, 30, 23, 0)
# Get available data
ctx = {'coop': 'yes', 'csector': 'IA', 't': 'state', 'sz': 40, 'z': 'yes', 'f': 'linear', 'v': 'snow'}
df = load_data(ctx, sts, ets)
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx)
# add zeros and QC
df = add_zeros(df, ctx)

In [11]:
def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s from NWS COOP, LSR, CoCoRaHS Reports; "
                  "IEM 2020-2021 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
        twitter=True,
    )
    mp.contourf(
        lons, lats, vals, np.array([0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]), cmap=nwssnow(), clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df_useme_plot)
    mp.drawcounties()
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'],
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=1)
    return mp

In [25]:
cull = ['JFFI4', 496, 9, 320, 322, 395, 66, 68, 'IA-SR-2', 'IA-PK-115', 'IA-PK-68', 'IA-SR-12', 53, 49, 39]
if cull:
    df.loc[df['nwsli'].isin(cull), USEME] = False
# df.loc[df[df["state"] == "MN"].index, USEME] = False
hardcode = [
    ['CKPI4', 4],
    ['CINI4', 4],
    ['IA-PA-5', 4],
    ['MSNI4', 4],
]
for nwsli, val in hardcode:
    df.loc[df['nwsli'] == nwsli, 'val'] = val

ctx['csector'] = "IA"
mp = workflow(ctx, df, isfinal=True, lower=6, upper=10)
res = mp.postprocess(filename='210116.png')
mp.close()

     state  wfo  val         lon        lat                               geo  \
2       VT  ALY  9.0  -72.880000  42.870000    POINT (2159893.731 128078.312)   
3       NY  ALY  9.0  -74.330000  42.300000     POINT (2068711.290 29128.630)   
4       AK  AJK  9.0 -135.450000  59.240000  POINT (-1937805.026 2044889.191)   
5       VT  ALY  8.7  -73.020000  42.820000    POINT (2151047.912 118997.703)   
6       VT  ALY  8.5  -72.780000  42.980000    POINT (2163518.675 142498.924)   
...    ...  ...  ...         ...        ...                               ...   
1037    IA  DMX  6.0  -94.616700  42.400000    POINT (441754.058 -274563.300)   
1038    IA  DMX  6.0  -94.990800  42.423900    POINT (410946.142 -273857.425)   
1039    IA  DMX  6.0  -93.799864  41.783269    POINT (513678.738 -338241.280)   
1040    IA  DMX  6.0  -94.965020  41.494340    POINT (419220.955 -376916.846)   
1041    IA  DMX  6.0  -95.003100  41.418100    POINT (416553.379 -385572.801)   

      used_for_analysis    